# Process PDF Documents

> This notebook will be used to process the pdf files exported from WikiVoyage and store them in a local folder.

In [1]:
import hashlib
import os
import pickle

from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)
from marker.convert import convert_single_pdf
from marker.logger import configure_logging
from marker.models import load_all_models
from tqdm import tqdm

/home/ec2-user/SageMaker/ethical-travel-recommender/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download files and process pdf files with marker converter (.pdf -> .md)

https://github.com/VikParuchuri/marker?tab=readme-ov-file

In [2]:
def parse_file_name(filename):
    # Remove space and dots on filename
    if filename.endswith(".pdf"):
        name_part = filename[:-4]
        name_part = name_part.replace(" ", "").replace(".", "")
        cleaned_filename = name_part + ".pdf"
    else:
        cleaned_filename = filename.replace(" ", "").replace(".", "")
    return cleaned_filename

In [3]:
def convert_pdf_to_markdown(fname, reference_folder, model_lst, md_out_path=None):

    md_filename = fname.rsplit(".", 1)[0] + ".md"

    pdf_filename = os.path.join(reference_folder, fname)

    print(pdf_filename)
    full_text, _, _ = convert_single_pdf(pdf_filename, model_lst, batch_multiplier=1)

    if md_out_path:
        with open(os.path.join(md_out_path, md_filename), "w+") as f:
            f.write(full_text)
    else:
        return full_text

In [4]:
def download_and_process_pdf_file(
    f_key, text_splitter, markdown_splitter, model_lst, reference_folder="../data/"
):

    temp_file_name = parse_file_name(f_key)
    temp_file_path = os.path.join(reference_folder, temp_file_name)

    mdfile = convert_pdf_to_markdown(temp_file_name, reference_folder, model_lst, None)

    md_header_split = markdown_splitter.split_text(mdfile)

    documents = []
    for split in md_header_split:

        split_texts = text_splitter.split_text(split.page_content)

        for i, split_text in enumerate(split_texts):

            document_id = f"{f_key}_part_{i}"
            hash_object = hashlib.md5(document_id.encode())
            hash_hex = hash_object.hexdigest()
            document_id = hash_hex[:10]

            metadata_dict = {
                "document_id": document_id,
                "pdf_name": f_key,
                "pdf_part": i,
            }

            metadata_dict.update(split.metadata)

            documents.append({"metadata": metadata_dict, "content": split_text})

    return documents

List files in folder

In [5]:
import os


def list_pdf_files(directory_path):
    """
    Lists all PDF files in the given directory and attempts to read them as binary data.

    Parameters:
        directory_path (str): The path to the directory containing PDF files.

    Returns:
        dict: A dictionary where keys are filenames and values are raw binary content or text content (if readable).
    """
    pdf_files_content = {}

    # Check if the directory exists
    if not os.path.isdir(directory_path):
        print(f"The directory '{directory_path}' does not exist.")
        return pdf_files_content

    filenames = [f for f in os.listdir(directory_path) if f.lower().endswith(".pdf")]

    return filenames


# Usage example
directory = "../data/"  # Replace with your directory path
filenames = list_pdf_files(directory)
print(f"Number of PDFs read: {len(filenames)}")

Number of PDFs read: 12


Setup marker class for converting pdf to markdown

In [6]:
configure_logging()
model_lst = load_all_models()

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype


Setup process splitters

In [7]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
chunk_size = 500
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [8]:
documents = []

for filename in tqdm(filenames):

    print("\n filename: {} \n".format(filename))

    splitted_doc = download_and_process_pdf_file(
        filename,
        text_splitter,
        markdown_splitter,
        model_lst,
        reference_folder="../data/",
    )

    documents.append(splitted_doc)

  0%|          | 0/12 [00:00<?, ?it/s]


 filename: Responsible_travel.pdf 

../data/Responsible_travel.pdf



Detecting bboxes: 100%|██████████| 2/2 [00:01<00:00,  1.60it/s]

Detecting bboxes: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]

  8%|▊         | 1/12 [00:07<01:27,  7.91s/it]


 filename: Thailand.pdf 

../data/Thailand.pdf



Detecting bboxes: 100%|██████████| 17/17 [00:10<00:00,  1.60it/s]

Detecting bboxes: 100%|██████████| 11/11 [00:13<00:00,  1.24s/it]

 17%|█▋        | 2/12 [01:15<07:10, 43.01s/it]

...


Detecting bboxes: 100%|██████████| 9/9 [00:05<00:00,  1.63it/s]

Detecting bboxes: 100%|██████████| 6/6 [00:07<00:00,  1.18s/it]

 83%|████████▎ | 10/12 [06:25<01:17, 38.65s/it]


 filename: Singapore.pdf 

../data/Singapore.pdf



Detecting bboxes: 100%|██████████| 20/20 [00:12<00:00,  1.61it/s]

Detecting bboxes: 100%|██████████| 13/13 [00:15<00:00,  1.23s/it]

 92%|█████████▏| 11/12 [07:49<00:52, 52.59s/it]


 filename: Indonesia.pdf 

../data/Indonesia.pdf



Detecting bboxes: 100%|██████████| 17/17 [00:11<00:00,  1.53it/s]

Detecting bboxes: 100%|██████████| 12/12 [00:14<00:00,  1.17s/it]

100%|██████████| 12/12 [09:03<00:00, 45.25s/it]


In [9]:
# save the documents object into a pickle file to avoid computing it again
with open("../data/docs_processed.pickle", "wb") as f:
    pickle.dump(documents, f)

In [10]:
len(documents)

12

In [12]:
flattened_list = [item for sublist in documents for item in sublist]

In [13]:
flattened_list[0]

{'metadata': {'document_id': 'f392923b41',
  'pdf_name': 'Responsible_travel.pdf',
  'pdf_part': 0,
  'Header 2': 'Responsible Travel'},
 'content': 'See Sustainable travel for the ecological and appropriate technology dimension of travel sustainability.'}

In [14]:
# save the documents object into a pickle file to avoid computing it again
with open("../data/docs_processed.pickle", "wb") as f:
    pickle.dump(flattened_list, f)